In [9]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np
import gc

from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# Load Data

In [3]:
data = pd.read_csv('/kaggle/input/diabetes-dataset/diabetes_final.csv')
data.drop('Unnamed: 0', axis=1, inplace=True)
data.head()

,age,physical_activity_minutes_per_week,diet_score,screen_time_hours_per_day,family_history_diabetes,hypertension_history,cardiovascular_history,bmi,waist_to_hip_ratio,systolic_bp,...,ethnicity_Other,ethnicity_White,employment_status_Employed,employment_status_Retired,employment_status_Student,employment_status_Unemployed,smoking_status_Current,smoking_status_Former,smoking_status_Never,diagnosed_diabetes
0,0.504956,1.138363,-0.165523,0.771162,-0.530172,-0.578582,-0.293278,1.362636,0.724256,1.274180,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1
1,-0.135884,0.285376,0.395977,1.095260,-0.530172,-0.578582,-0.293278,-0.700550,-1.197300,0.924138,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0
2,0.633124,-0.733470,0.227527,0.852187,1.886181,-0.578582,-0.293278,-0.951478,-0.983794,-0.055979,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1
3,1.530299,-0.828246,-1.456972,-0.322667,-0.530172,-0.578582,-0.293278,0.331043,0.510750,0.294063,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1
4,-0.264052,-0.117424,0.676727,-0.403691,-0.530172,-0.578582,-0.293278,-1.230287,-1.624313,-1.666172,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1


In [4]:
X = data.drop('diagnosed_diabetes', axis=1)
y = data['diagnosed_diabetes']

In [5]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


In [10]:
base_models = [
    ('lgbm', LGBMClassifier(n_estimators=1000, learning_rate=0.01, num_leaves=31, 
                            device='gpu', random_state=42, verbose=-1)),
    
 
    ('xgb', XGBClassifier(n_estimators=1000, learning_rate=0.01, max_depth=6, 
                          tree_method='hist', 
                          device='cuda',     
                          random_state=42)),
    
    ('cat', CatBoostClassifier(iterations=1000, learning_rate=0.03, depth=8, 
                               task_type="GPU", random_state=42, verbose=False))
]



In [11]:
stack_model = StackingClassifier(
    estimators=base_models,
    final_estimator=LogisticRegression(),
    cv=skf, 
    stack_method='predict_proba',
    n_jobs=1 
)

In [12]:
gc.collect()
print("Starting Stacking Training (This might take some time due to GPU processing)...")
stack_model.fit(X, y)

Starting Stacking Training (This might take some time due to GPU processing)...


/usr/local/lib/python3.12/dist-packages/xgboost/core.py:774: UserWarning: [01:05:45] WARNING: /workspace/src/common/error_msg.cc:41: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)


StackingClassifier(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                   estimators=[('lgbm',
                                LGBMClassifier(device='gpu', learning_rate=0.01,
                                               n_estimators=1000,
                                               random_state=42, verbose=-1)),
                               ('xgb',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              device='cuda',
                                              early_stopp...
                                              max_cat_to_onehot=None,
                                              max_delta_step=None, max_depth=6,
                                              max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=1000, n_jobs=None,
                                              num_parallel_tree=None, ...)),
                               ('cat',
                                <catboost.core.CatBoostClassifier object at 0x7822afdd15e0>)],
                   final_estimator=LogisticRegression(), n_jobs=1,
                   stack_method='predict_proba')

In [13]:
final_auc = roc_auc_score(y, stack_model.predict_proba(X)[:, 1])
print(f"Final Stacking Training AUC: {final_auc:.4f}")

Final Stacking Training AUC: 0.9705


In [14]:
import joblib

model_filename = 'final_stacking_diabetes_model.pkl'
joblib.dump(stack_model, model_filename, compress=3)

['final_stacking_diabetes_model.pkl']